In [1]:
import sys
sys.path.insert(0, "../../src/chung/")

import torch
from IPython.display import display

import utils
from models import S2STrans
utils.translation_root = '../../src/chung/data/translation'

## Translation All Around

This notebook aims to translate a sentence from English to German, from German to Italian, then from Italian to English using the trained models in this experiment.

First, let's get the datasets for their vocabulary:

In [2]:
datasets = {'en_de': utils.IWSLT2017TransDataset('en', 'de'),
            'de_it': utils.IWSLT2017TransDataset('de', 'it'),
            'it_en': utils.IWSLT2017TransDataset('it', 'en')}

building en vocab...
found previously built en vocab, loading...
there are 6079 tokens in en vocab
building de vocab...
found previously built de vocab, loading...
there are 6624 tokens in de vocab
building translation train en->de dataset...
found preprocessed train en->de dataset, loading...
taking 25000 subset of full data...
truncating all sequences to have max 50 tokens...
building de vocab...
found previously built de vocab, loading...
there are 6702 tokens in de vocab
building it vocab...
found previously built it vocab, loading...
there are 7447 tokens in it vocab
building translation train de->it dataset...
found preprocessed train de->it dataset, loading...
taking 25000 subset of full data...
truncating all sequences to have max 50 tokens...
building it vocab...
found previously built it vocab, loading...
there are 7426 tokens in it vocab
building en vocab...
found previously built en vocab, loading...
there are 6141 tokens in en vocab
building translation train it->en datase

Now, let's load each of the trained models:

In [3]:
models = {}
model_types = {'lstm': '49', 'gru': '49', 'tanh': '19'}
for dataset in datasets:
    for rnn_type in model_types:
        models[f'{rnn_type}_{dataset}'] = S2STrans(len(datasets[dataset].src_vocab),
                                                   len(datasets[dataset].tgt_vocab),
                                                   rnn_type=rnn_type,
                                                   num_layers=2)
        model_path = f"../../src/chung/exp/translation/{dataset}/{rnn_type}/{model_types[rnn_type]}/model.pt"
        models[f'{rnn_type}_{dataset}'].load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
        models[f'{rnn_type}_{dataset}'].eval()

And use the following code for translation:

In [4]:
def translate(dataset, rnn_type, sentence):
    display(f"Source: {sentence}")
    ids = torch.LongTensor(datasets[dataset].convert_src_text_to_ids(sentence)).unsqueeze(1)
    pred, pred_score = models[f'{rnn_type}_{dataset}'].infer(ids)
    pred = pred.squeeze(1).tolist()
    pred_score = pred_score.item()
    pred_text = datasets[dataset].convert_tgt_ids_to_text(pred)
    display(f"Pred: {pred_text}")
    display(f"Pred NLL Score: {pred_score}")
    return pred_text

Let this sentence be the example we'll use throughout all models:

In [5]:
sentence = "What starts here changes the world?"

In [6]:
processed_ids = datasets['en_de'].convert_src_text_to_ids(sentence)
processed = datasets['en_de'].convert_src_ids_to_text(processed_ids)
display(f"Original: {sentence}")
display(f"Processed: {processed}")

'Original: What starts here changes the world?'

'Processed: What starts here changes the world ?'

### LSTM

In [7]:
pred_text = translate('en_de', 'lstm', processed)

'Source: What starts here changes the world ?'

'Pred: Was passiert hier die Welt ?'

'Pred NLL Score: 3.308420419692993'

In [8]:
pred_text = translate('de_it', 'lstm', pred_text)

'Source: Was passiert hier die Welt ?'

'Pred: Cosa succede nel mondo ?'

'Pred NLL Score: 5.138217926025391'

In [9]:
pred_text = translate('it_en', 'lstm', pred_text)

'Source: Cosa succede nel mondo ?'

'Pred: What happens in the world ?'

'Pred NLL Score: 4.024959564208984'

### GRU

In [10]:
pred_text = translate('en_de', 'gru', processed)

'Source: What starts here changes the world ?'

'Pred: Was passiert hier den Menschen ?'

'Pred NLL Score: 6.731745719909668'

In [11]:
pred_text = translate('de_it', 'gru', pred_text)

'Source: Was passiert hier den Menschen ?'

'Pred: Cosa significa questo ?'

'Pred NLL Score: 4.70180082321167'

In [12]:
pred_text = translate('it_en', 'gru', pred_text)

'Source: Cosa significa questo ?'

'Pred: What does that mean ?'

'Pred NLL Score: 1.6696586608886719'

Honestly though, what does that mean? What does changing the world have to do with what does that mean?

### Tanh

In [13]:
pred_text = translate('en_de', 'tanh', processed)

'Source: What starts here changes the world ?'

'Pred: Was ist die erste Frage ?'

'Pred NLL Score: 8.269089698791504'

In [14]:
pred_text = translate('de_it', 'tanh', pred_text)

'Source: Was ist die erste Frage ?'

'Pred: Qual è il primo esempio .'

'Pred NLL Score: 8.318471908569336'

In [15]:
pred_text = translate('it_en', 'tanh', pred_text)

'Source: Qual è il primo esempio .'

"Pred: What 's the right now ."

'Pred NLL Score: 7.769245624542236'

Yeah, I think LSTM did the best here.